In [1]:
# Libraries 

import psycopg2
import psycopg2.extras

# PostgreSQL

## Connect to Database

In [3]:
hostname = 'localhost'
database = 'sales'
username = 'postgres'
pwd = 1984
port_id = 5432

conn = psycopg2.connect(
    host = hostname,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id)

cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

## Look at all Table Names

In [4]:
query = "SELECT * FROM information_schema.tables WHERE table_schema = 'public'"
cur.execute(query)
for record in cur.fetchall():
    print(record)

['sales', 'public', 'employee', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]
['sales', 'public', 'v2', 'VIEW', None, None, None, None, None, 'NO', 'NO', None]
['sales', 'public', 'web_events', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]
['sales', 'public', 'sales_reps', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]
['sales', 'public', 'region', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]
['sales', 'public', 'orders', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]
['sales', 'public', 'accounts', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None]


## General Look all Tables

### ERD - Entity Relationship Diagrams

![ERD](01.png)

### web_events

In [5]:
query = 'SELECT * FROM web_events LIMIT 5'
cur.execute(query)
for record in cur.fetchall():
    print(record)

[1, 1001, datetime.datetime(2015, 10, 6, 17, 13, 58), 'direct']
[2, 1001, datetime.datetime(2015, 11, 5, 3, 8, 26), 'direct']
[3, 1001, datetime.datetime(2015, 12, 4, 3, 57, 24), 'direct']
[4, 1001, datetime.datetime(2016, 1, 2, 0, 55, 3), 'direct']
[5, 1001, datetime.datetime(2016, 2, 1, 19, 2, 33), 'direct']


### accounts

In [6]:
query = 'SELECT * FROM accounts LIMIT 5'
cur.execute(query)
for record in cur.fetchall():
    print(record)

[1001, 'Walmart', 'www.walmart.com', Decimal('40.23849561'), Decimal('-75.10329704'), 'Tamara Tuma', 321500]
[1011, 'Exxon Mobil', 'www.exxonmobil.com', Decimal('41.16915630'), Decimal('-73.84937379'), 'Sung Shields', 321510]
[1021, 'Apple', 'www.apple.com', Decimal('42.29049481'), Decimal('-76.08400942'), 'Jodee Lupo', 321520]
[1031, 'Berkshire Hathaway', 'www.berkshirehathaway.com', Decimal('40.94902131'), Decimal('-75.76389759'), 'Serafina Banda', 321530]
[1041, 'McKesson', 'www.mckesson.com', Decimal('42.21709326'), Decimal('-75.28499823'), 'Angeles Crusoe', 321540]


### orders 

In [7]:
query = 'SELECT * FROM orders LIMIT 5'
cur.execute(query)
for record in cur.fetchall():
    print(record)

[1, 1001, datetime.datetime(2015, 10, 6, 17, 31, 14), 123, 22, 24, 169, Decimal('613.77'), Decimal('164.78'), Decimal('194.88'), Decimal('973.43')]
[2, 1001, datetime.datetime(2015, 11, 5, 3, 34, 33), 190, 41, 57, 288, Decimal('948.10'), Decimal('307.09'), Decimal('462.84'), Decimal('1718.03')]
[3, 1001, datetime.datetime(2015, 12, 4, 4, 21, 55), 85, 47, 0, 132, Decimal('424.15'), Decimal('352.03'), Decimal('0.00'), Decimal('776.18')]
[4, 1001, datetime.datetime(2016, 1, 2, 1, 18, 24), 144, 32, 0, 176, Decimal('718.56'), Decimal('239.68'), Decimal('0.00'), Decimal('958.24')]
[5, 1001, datetime.datetime(2016, 2, 1, 19, 27, 27), 108, 29, 28, 165, Decimal('538.92'), Decimal('217.21'), Decimal('227.36'), Decimal('983.49')]


### sales_reps

In [8]:
query = 'SELECT * FROM sales_reps LIMIT 5'
cur.execute(query)
for record in cur.fetchall():
    print(record)

[321500, 'Samuel Racine', 1]
[321510, 'Eugena Esser', 1]
[321520, 'Michel Averette', 1]
[321530, 'Renetta Carew', 1]
[321540, 'Cara Clarke', 1]


### region

In [9]:
query = 'SELECT * FROM region LIMIT 5'
cur.execute(query)
for record in cur.fetchall():
    print(record)

[1, 'Northeast']
[2, 'Midwest']
[3, 'Southeast']
[4, 'West']


## Subqueries

In [15]:
# What is the top channel used by each account to market products?
# How often was that same channel used?

SELECT t3.id, t3.name, t3.channel, t3.ct
FROM (SELECT a.id, a.name, we.channel, COUNT(*) ct
     FROM accounts a
     JOIN web_events we
     On a.id = we.account_id
     GROUP BY a.id, a.name, we.channel) T3
JOIN (SELECT t1.id, t1.name, MAX(ct) max_chan
      FROM (SELECT a.id, a.name, we.channel, COUNT(*) ct
            FROM accounts a
            JOIN web_events we
            ON a.id = we.account_id
            GROUP BY a.id, a.name, we.channel) t1
      GROUP BY t1.id, t1.name) t2
ON t2.id = t3.id AND t2.max_chan = t3.ct
ORDER BY t3.id;


query = "   SELECT t3.id, t3.name, t3.channel, t3.ct                     \
            FROM (SELECT a.id, a.name, we.channel, COUNT(*) ct           \
                 FROM accounts a                                         \
                 JOIN web_events we                                      \
                 On a.id = we.account_id                                 \
                 GROUP BY a.id, a.name, we.channel) T3                   \
            JOIN (SELECT t1.id, t1.name, MAX(ct) max_chan                \
                  FROM (SELECT a.id, a.name, we.channel, COUNT(*) ct     \
                        FROM accounts a                                  \
                        JOIN web_events we                               \
                        ON a.id = we.account_id                          \
                        GROUP BY a.id, a.name, we.channel) t1            \
                  GROUP BY t1.id, t1.name) t2                            \
            ON t2.id = t3.id AND t2.max_chan = t3.ct                     \
            ORDER BY t3.id                                               \
            LIMIT 5                                                      \
"
cur.execute(query)
for record in cur.fetchall():
    print(record)

[1001, 'Walmart', 'direct', 22]
[1011, 'Exxon Mobil', 'facebook', 1]
[1011, 'Exxon Mobil', 'direct', 1]
[1011, 'Exxon Mobil', 'adwords', 1]
[1021, 'Apple', 'direct', 9]


In [ ]:
## 1. Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales. 

## -- SOLUTION 01.01
WITH t2 AS(	SELECT  s.region_id, s.name s_name,  MAX(t1.max_amt) max_amt
      FROM (SELECT a.sales_rep_id s_id, SUM(o.total_amt_usd) max_amt
            FROM accounts a
            JOIN orders o ON a.id = o.account_id
            GROUP BY sales_rep_id
            ORDER BY 2 DESC) t1
     JOIN sales_reps s ON t1.s_id = s.id
     GROUP BY s.region_id, s.name),
     
t3 AS ( SELECT  s.region_id, MAX(t4.max_amt) max_amt
        FROM (SELECT a.sales_rep_id s_id, SUM(o.total_amt_usd) max_amt
              FROM accounts a
              JOIN orders o ON a.id = o.account_id
              GROUP BY a.sales_rep_id
              ORDER BY 2 DESC) t4
        JOIN sales_reps s ON t4.s_id = s.id
        GROUP BY s.region_id)

SELECT   r.name, t2.s_name, t3.max_amt
FROM t3
LEFT JOIN t2 ON t3.region_id = t2.region_id AND t3.max_amt = t2.max_amt
LEFT JOIN region r ON r.id = t3.region_id
ORDER BY 3 DESC


## -- SOLUTION 01.02
WITH t1 AS (
   SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
   FROM sales_reps s
   JOIN accounts a
   ON a.sales_rep_id = s.id
   JOIN orders o
   ON o.account_id = a.id
   JOIN region r
   ON r.id = s.region_id
   GROUP BY 1,2
   ORDER BY 3 DESC), 
t2 AS (
   SELECT region_name, MAX(total_amt) total_amt
   FROM t1
   GROUP BY 1)
SELECT t1.rep_name, t1.region_name, t1.total_amt
FROM t1
JOIN t2
ON t1.region_name = t2.region_name AND t1.total_amt = t2.total_amt;

In [ ]:
##    2. For the region with the largest sales total_amt_usd, how many total orders were placed? 

WITH t1 as (
    SELECT r.name r_name, SUM(total_amt_usd) sum_amt, COUNT(o.id) tt_orders
    FROM sales_reps s 
    JOIN accounts a ON s.id = a.sales_rep_id
    JOIN orders o ON a.id = o.account_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    ORDER BY 2 DESC)

SELECT *
FROM t1
LIMIT 1

In [ ]:
##    2. For the region with the largest sales total_amt_usd, how many total orders were placed? 

WITH t1 as (
    SELECT r.name r_name, SUM(total_amt_usd) sum_amt, COUNT(o.id) tt_orders
    FROM sales_reps s 
    JOIN accounts a ON s.id = a.sales_rep_id
    JOIN orders o ON a.id = o.account_id
    JOIN region r ON r.id = s.region_id
    GROUP BY 1
    ORDER BY 2 DESC)

SELECT *
FROM t1
LIMIT 1